In [1]:
import sys, time, pickle
import torch

sys.path.insert(0, '../../Utils')
from global_constants import gpu_device

sys.path.insert(0, '../')
from BERT_models import BERT_SST2_MODEL

from _utils import sample_random_glue_sst2, get_continuation_mapping, \
                    get_continuous_attributions, get_continuous_raw_inputs, \
                    collect_info_for_metric, save_info, download_HTML

In [2]:
sst2_data_raw, targets, idxs = sample_random_glue_sst2()

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3b24abff24d1d8c0.arrow
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-5960909ab3834668.arrow


In [3]:
model = BERT_SST2_MODEL()
tokenizer = model.tokenizer

In [4]:
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [5]:
from captum.attr import GuidedBackprop
from captum.attr import visualization 

In [6]:
gbp  = GuidedBackprop(model)

In [7]:
def generate_record(raw_review, target):
    #tokenizer operations
    tokenized = tokenizer(raw_review, truncation=True, return_offsets_mapping=True)
    offset_mapping = tokenized['offset_mapping']
    conti_map = get_continuation_mapping(offset_mapping)
    input_ids = torch.tensor(tokenized['input_ids']).unsqueeze(0).to(gpu_device)
    detokenized = [t.replace('#', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]
    
    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()
    
    #categorizing results
    pred_class = 'Pos' if pred_prob > 0.5 else 'Neg' 
    true_class = 'Pos' if target > 0.5 else 'Neg' 
    
    #attribution algorithm working
    attribution = gbp.attribute(input_emb)
    word_attributions = attribution.squeeze(0).sum(dim=1)
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'Pos' if attr_score > 0.5 else 'Neg'
    convergence_score = None
    
    
    #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)

#     print(f'word attributions {word_attributions}')
#     print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
#     collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=word_attributions,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input_ids=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [8]:
for i, (datum_raw, target) in enumerate(zip(sst2_data_raw, targets), start=1):
    print(f'Raw review: {datum_raw}')
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record]))


Raw review: its oscar nomination 
GT target: 1
word attr tensor([-0.6134,  0.3798,  0.3681,  0.4772, -0.3409], device='cuda:0')
conti attr [tensor(-0.6134, device='cuda:0'), tensor(0.3798, device='cuda:0'), tensor(0.3681, device='cuda:0'), tensor(0.4772, device='cuda:0'), tensor(-0.3409, device='cuda:0')]
detokenized ['[CLS]', 'its', 'oscar', 'nomination', '[SEP]']
len conti_raw 5
conti_raw ['[CLS]', 'its', 'oscar', 'nomination', '[SEP]']


/home/user/anaconda3/envs/metric/lib/python3.9/site-packages/captum/attr/_core/guided_backprop_deconvnet.py:64: UserWarning: Setting backward hooks on ReLU activations.The hooks will be removed after the attribution is finished
  warnings.warn(


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Pos,Pos (0.96),Neg,0.27,[CLS] its oscar nomination [SEP]


<IPython.core.display.HTML object>
Raw review: shenanigans and slapstick 
GT target: 1
word attr tensor([-0.0973, -0.1058,  0.2980, -0.1760,  0.2896,  0.0049,  0.6370,  0.6079,
         0.0207], device='cuda:0')
conti attr [tensor(-0.0973, device='cuda:0'), tensor(0.1248, device='cuda:0'), tensor(0.0049, device='cuda:0'), tensor(0.6224, device='cuda:0'), tensor(0.0207, device='cuda:0')]
detokenized ['[CLS]', 'shen', 'ani', 'gan', 's', 'and', 'slap', 'stick', '[SEP]']
len conti_raw 5
conti_raw ['[CLS]', 'shenanigans', 'and', 'slapstick', '[SEP]']


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Pos,Neg (0.03),Pos,1.48,[CLS] shenanigans and slapstick [SEP]


...

<IPython.core.display.HTML object>
Raw review: unrewarding 
GT target: 0
word attr tensor([-0.5483, -0.1399, -0.2521, -0.6928,  0.3691], device='cuda:0')
conti attr [tensor(-0.5483, device='cuda:0'), tensor(-0.4444, device='cuda:0'), tensor(0.3691, device='cuda:0')]
detokenized ['[CLS]', 'un', 'rew', 'arding', '[SEP]']
len conti_raw 3
conti_raw ['[CLS]', 'unrewarding', '[SEP]']


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Neg,Neg (0.14),Neg,-1.26,[CLS] unrewarding [SEP]


<IPython.core.display.HTML object>
Raw review: old-hat 
GT target: 0
word attr tensor([-0.1731, -0.5756, -0.2757, -0.4593, -0.5930], device='cuda:0')
conti attr [tensor(-0.1731, device='cuda:0'), tensor(-0.4425, device='cuda:0'), tensor(-0.5930, device='cuda:0')]
detokenized ['[CLS]', 'old', '-', 'hat', '[SEP]']
len conti_raw 3
conti_raw ['[CLS]', 'old-hat', '[SEP]']


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Neg,Neg (0.05),Neg,-2.08,[CLS] old-hat [SEP]


<IPython.core.display.HTML object>


In [9]:
file_name_base = 'Guided_Backprop'

In [10]:
save_info(idxs, sst2_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, 
          fname=f'../MethodOutputs/{file_name_base}_out.pkl')

{'indices': [27798,
  28244,
  48083,
  8795,
  91,
  20682,
  59890,
  41763,
  19795,
  9957,
  28334,
  55664,
  16438,
  18031,
  9297,
  66913,
  9289,
  6789,
  25294,
  33294,
  29699,
  41960,
  6554,
  8772,
  17921,
  8027,
  49091,
  54186,
  15080,
  22382,
  23856,
  9464,
  23831,
  59434,
  17862,
  24584,
  26216,
  63094,
  55787,
  3993,
  25463,
  17540,
  51128,
  46224,
  32656,
  30105,
  28646,
  17011,
  7812,
  48236],
 'raw_data': ['its oscar nomination ',
  'shenanigans and slapstick ',
  'an unsettling sight , ',
  'the climactic hourlong cricket match ',
  'alternating between facetious comic parody and pulp melodrama , this smart-aleck movie ... tosses around some intriguing questions about the difference between human and android life ',
  'to be a part of that elusive adult world ',
  'emotional power ',
  'reminds you of why animation is such a perfect medium for children , because of the way it allows the mind to enter and accept another world ',
  'un

In [11]:
download_HTML(cur_file_name=f'{file_name_base}_SST2_BERT.ipynb',
              out_file_name=f'{file_name_base}_SST2_BERT.html')